In [14]:
from google.colab import files
fread = lambda path: open(path,'r').read()

In [ ]:
#@title Day 1: Secret Entrance
import math

inputs = fread('input.txt').splitlines()

def rotate(pos, d, rotation, part1, part2):
  flip = lambda p : [p%100,(100-p)%100][d=='R']
  pos = flip(pos) + rotation
  return (flip(pos), part1 + (flip(pos) == 0), part2 + math.floor(pos / 100))

acc = (50, 0, 0)
[acc := rotate(acc[0], i[0],int(i[1:]), acc[1], acc[2]) for i in inputs]

print(acc[1])
print(acc[2])

In [ ]:
#@title Day 2: Gift Shop
import math, re

inputs = fread('input 2.txt')
inputs = [list(map(int,s.split('-'))) for s in inputs.split(',')]

def bads(lower, upper, pattern):
  some = [i for i in range(lower, upper+1) if re.search(pattern, str(i))]
  return sum([s for s in some if s >= lower and s <= upper])

print(sum([bads(*i,r'^([0-9]+)\1$') for i in inputs]))
print(sum([bads(*i,r'^([0-9]+)\1+$') for i in inputs]))

In [ ]:
#@title Day 3: Lobby
from itertools import accumulate
import math

inputs = fread('input 3.txt')
inputs = [list(map(int,s)) for s in inputs.splitlines()]

def joltage(line, d):
  def best_idx(prev,j):
    offset = [0,prev+1][prev > -1]
    values = line[offset:len(line)-(d-1-j)]
    return max(range(len(values)), key=values.__getitem__)+offset

  idxs = list(accumulate(range(-1,d), best_idx))[1:]
  return sum([int(math.pow(10,d-1-j)) * line[idxs[j]] for j in range(d)])

print(sum(map(lambda l: joltage(l, 2), inputs)))
print(sum(map(lambda l: joltage(l, 12), inputs)))

In [ ]:
#@title Day 4: Printing Department

inputs = """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@."""
inputs = fread('input 4.txt')
inputs = inputs.splitlines()

max_x = len(inputs[0])
max_y = len(inputs)
is_tp = lambda a,x,y: 0 <= x < max_x and 0 <= y < max_y and a[y][x] == '@'

def prune_tp(prev):
  next = []
  count = 0

  for y in range(len(prev)):
    next.append([])

    for x in range(len(prev[0])):
      if not is_tp(prev,x,y):
        next[y].append(prev[y][x])
        continue

      ct = sum([is_tp(prev,i,j) for i in range(x-1,x+2) for j in range(y-1,y+2) if x != i or y != j])

      next[y].append('x' if ct < 4 else '@')
      count += ct < 4

  return (count, next)

def prune_all_tp():
  cont = True
  curr = inputs
  total = 0
  while cont:
    (removed, curr) = prune_tp(curr)
    total += removed
    cont = removed != 0
  return (total, curr)

print(prune_tp(inputs)[0])
print(prune_all_tp()[0])




In [ ]:
#@title Day 5-1: Cafeteria
inputs = [s.splitlines() for s in fread('input 5.txt').split('\n\n')];
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ids = list(map(int,inputs[1]))
count = sum([any(low <= id <= high for (low,high) in ranges) for id in ids])
print(count)

In [ ]:
#@title Day 5-2: Cafeteria
inputs = fread('input 5.txt')

inputs = [s.splitlines() for s in inputs.split('\n\n')]
ranges = [list(map(int,s.split('-'))) for s in inputs[0]]
ranges = sorted(ranges, key=lambda r: r[0])

def insert(sorted_list, new_element, i, len):
  if(i >= len):
    return sorted_list + [new_element];
  if(sorted_list[i][0] > new_element[1]):
    return sorted_list[:i] + [new_element] + sorted_list[i:]
  if(sorted_list[i][1] < new_element[0]):
    return insert(sorted_list, new_element, i+1, len)
  return sorted_list[:i] + [[
      min(sorted_list[i][0],new_element[0]),
      max(sorted_list[i][1],new_element[1])
      ]] + sorted_list[i+1:]

merged_ranges = []
for i in range(len(ranges)):
  merged_ranges = insert(merged_ranges, ranges[i], 0, len(merged_ranges))

count = sum([high+1-low for (low,high) in merged_ranges])
print(count)

In [ ]:
#@title Day 6: Trash Compactor
from functools import reduce
from operator import add,mul

inputs = fread('input 6.txt')
inputs = inputs.splitlines()

def total(ops, ns): return sum([reduce(o, ns[i]) for i,o in enumerate(ops)])

# Part 1
strs = [s.split() for s in inputs]
operations = [[mul,add][c=='+'] for c in strs[-1]]
nums = [[int(strs[y][x]) for y in range(len(strs)-1)] for x in range(len(strs[0]))]
print(total(operations,nums))

# Part 2
operations = operations[::-1]
strs = [''.join([inputs[y][x] for y in range(len(inputs)-1)]).strip() for x in range(len(inputs[0])-1,-1,-1)]
strs = '\n'.join(strs).split('\n\n')
nums = [list(map(int, s.splitlines())) for s in strs]
print(total(operations, nums))

In [ ]:
#@title Day 7: Laboratories

inputs = fread("input 7.txt")
inputs = inputs.splitlines()

def split(ct, f, s):
  d = {i: 0 for i in range(-1,len(s)+1)}
  for i,c in enumerate(s):
    for j in ([i],[i-1,i+1])[c=='^']: d[j] += f[i]
  return (ct + sum([c=='^' and f[i]>0 for i,c in enumerate(s)]),
   [d[i] for i in range(len(f))])

state = (0, [int(c == 'S') for c in inputs[0]])
[state := split(*state, inputs[i]) for i in range(1,len(inputs))]

print(state[0])
print(sum(state[1]))

In [ ]:
#@title Day 8: Playground
import numpy as np
import math

inputs = fread("input 8.txt")

dist = lambda a,b: np.linalg.norm(a-b)
find_parent= lambda a,ps: a if a==ps[a] else find_parent(ps[a],ps)

def connect(i, j, sets, sizes, n_sets):
  ps = [find_parent(i,sets),find_parent(j,sets)]
  if ps[0] == ps[1]: return (sets, sizes, n_sets)

  leader = min(*ps)
  size = sum([sizes[p] for p in ps])
  for p in ps: sizes[p],sets[p] = 0,leader
  sizes[leader] = size
  return (sets, sizes, n_sets-1)

def solve(jns, n):
  dists = {(i,j): dist(jn,jns[j]) for i,jn in enumerate(jns) for j in range(i+1,len(jns))}
  ordered_dists = sorted(dists, key=lambda k: dists[k])

  n_sets = len(jns)
  sets = list(range(n_sets))
  sizes = [1]*n_sets

  # part 1
  for i,j in ordered_dists[0:n]: (sets,sizes,n_sets) = connect(i,j, sets, sizes, n_sets)
  ordered_sizes = sorted([s for s in sizes if s > 0], reverse=True)
  yield math.prod(ordered_sizes[0:3])

  # part 2
  for i,j in ordered_dists[n:]:
    (sets,sizes,n_sets) = connect(i,j,sets,sizes,n_sets)

    if(n_sets==1):
      yield jns[i][0]*jns[j][0]
      break;

jns = [np.array(list(map(int, j.split(',')))) for j in inputs.splitlines()]
print(*solve(jns,1000))

In [ ]:
#@title Day 9: Movie Theater
import numpy as np

inputs = fread("input 9.txt")
inputs = [list(map(int,s.split(','))) for s in inputs.splitlines()]

n = len(inputs)
def get_area(p,q): return abs((q[0]+1-p[0])*(q[1]+1-p[1]))

# Part 1
area = max([get_area(i,j) for i in inputs for j in inputs])
print(area)

# Part 2
def top_left(r0,r1): return (min(r0[0],r1[0]),min(r0[1],r1[1]))
def bottom_right(r0,r1): return (max(r0[0],r1[0]),max(r0[1],r1[1]))
def intersects(tl,br,s):
  return not(s[0][0]>=br[0] and s[1][0]>=br[0]) \
    and not(s[0][0]<=tl[0] and s[1][0]<=tl[0]) \
    and not(s[0][1]>=br[1] and s[1][1]>=br[1]) \
    and not(s[1][1]<=tl[1] and s[1][1]<=tl[1])

rays = [[inputs[(i+1)%n],inputs[i]] for i in range(n)]
rects = [(top_left(i,j),bottom_right(i,j)) for a,i in enumerate(inputs[0:n-1]) for j in inputs[a+1:]]
rects = [r for r in rects if r[0][0]!=r[1][0] and r[0][1]!=r[1][1]]
area = max([get_area(i,j) for i,j in rects if not any(intersects(i,j,r) for r in rays)])
print(area)

In [2]:
#@title Day 10-1: Factory

inputs = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""
inputs = fread('input 10.txt')

class Machine:
  def __init__(self, goal, n_lights, buttons):
    self.__goal = goal
    self.__n_lights = n_lights
    self.__buttons = buttons
    self.__n_buttons = len(buttons)

  def parse(l):
    input = l.split(' ')
    goal = 0
    n_lights = len(input[0][1:-1])
    [goal := (goal << 1) | (c=='#') for c in input[0][1:-1]]
    buttons = [sum([1<<(n_lights-1-n) for n in map(int,s[1:-1].split(","))]) for s in input[1:-1]]
    return Machine(goal, n_lights, buttons)

  def solve(self, i_b, states):
    if i_b>=self.__n_buttons: return states[self.__goal]
    for s,s_presses in states.copy().items():
      t = self.__buttons[i_b]^s
      t_presses = s_presses+1
      states[t] = min(states[t], t_presses) if t in states else t_presses
    return self.solve(i_b+1,states)

ms = [Machine.parse(s) for s in inputs.splitlines()]
print(sum([m.solve(0, {0:0}) for m in ms]))

375


In [ ]:
#@title Day 10-2: Factory
import numpy as np

inputs = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""
inputs = fread('input 10.txt')

class Machine:
  def __init__(self, goal, n_lights, buttons):
    self.__goal = goal
    self.__n_lights = n_lights
    self.__buttons = buttons
    self.__n_buttons = len(buttons)
    # print(goal,buttons)

  def parse(l):
    input = l.split(' ')
    n_lights = len(input[0][1:-1])
    buttons = np.array([[int(i in set(map(int,s[1:-1].split(",")))) for i in range(n_lights)] for s in input[1:-1]])
    goal = np.array([int(c) for c in input[-1][1:-1].split(',')])
    return Machine(goal, n_lights, buttons[0:])

  def part2(self):
    tostr = lambda arr: ','.join(map(str,arr))
    clicks = 0
    search = [self.__goal]
    seen = {tostr(self.__goal.tolist()): 0}

    while True:
      next = []
      clicks += 1
      for state in search:
        for b in self.__buttons:
          curr = state - b
          s = tostr(curr)
          # print (curr)
          if s in seen: continue
          if any(n < 0 for n in curr): continue
          if all(n == 0 for n in curr): return clicks
          next.append(curr)
          seen[s] = clicks
      search = next
      if not any(True for n in next): return None

ms = [Machine.parse(s) for s in inputs.splitlines()]

sum = 0
print("TOTAL:",len(ms))
for i,m in enumerate(ms):
  sum += m.part2()
  print(i,end=' ')
print(sum)

In [16]:
#@title Day 11

inputs = """aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out"""
inputs = fread('input 11.txt')

inputs = [s.split() for s in inputs.splitlines()]
outs = {ns[0][:-1]: set(ns[1:]) for ns in inputs}

nodes = set()
[nodes := nodes | outs[n] | {n} for n in outs.keys()]

def get_ways(src, tar):
  ways = {n: 0 for n in nodes}
  ways[src] = 1
  seen = set()
  to_visit = [src]
  while any(to_visit):
    curr, to_visit = to_visit[0],to_visit[1:]
    if curr in outs:
      for n in outs[curr]:
        ways[n] += 1
        to_visit.append(n)
    seen |= {curr}
  return ways[tar]

print(get_ways('you','out'))


534
